In [1]:
import os
import sys
import django

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "True"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "LabTimetablingAPI.settings")
django.setup()

In [2]:
config = {
    "semester": 1,
    "local_search": {
        "algorithm": "simulated_annealing",
        "config": {
            "neighborhood": {
                "algorithm": "random_swap",
                "random_swap": {
                    "neighborhood_size": 100
                },
                "random_range_swap": {
                    "neighborhood_size_factor": 0.1,
                    "range_size_factor": 0.1
                },
                "distance_swap": {
                    "distance_percentage": 0.1
                },
                "swap": False
            },
            "simulated_annealing": {
                "initial_temperature": 100,
                "cooling_rate": 0.1,
                "max_iteration": 1000,
                "max_time": 60
            },
            "tabu_search": {
                "tabu_list_size": 50,
                "max_iteration": 1000,
                "max_time": 60,
                "max_iteration_without_improvement": 100,
                "max_time_without_improvement": 5
            }
        }
    },
    "algorithm": {
        "main": "genetic_local_search",
        "config": {
            "fitness": {
                "group_assignment_conflict": {
                    "max_threshold": 3,
                    "conflict_penalty": 1
                },
                "assistant_distribution": {
                    "max_group_threshold": 15,
                    "max_shift_threshold": 50,
                    "group_penalty": 1,
                    "shift_penalty": 1
                }
            },
            "max_iteration": 500,
            "population_size": 25,
            "elitism_size": 2,
            "operator": {
                "selection": {
                    "roulette_wheel": True,
                    "tournament": False,
                    "elitism": True,
                    "tournament_size": 5
                },
                "crossover": {
                    "single_point": True,
                    "two_point": False,
                    "uniform": False,
                    "crossover_probability": 0.1,
                    "uniform_probability": 0.5
                },
                "mutation": {
                    "swap": True,
                    "shift": False,
                    "random": False,
                    "mutation_probability": 0.1
                },
                "repair": {
                    "time_slot": True
                }
            }
        }
    }
}


In [3]:
import cProfile
from scheduling_algorithm.config_schema import ScheduleConfiguration
from scheduling_algorithm.utils.solution_generator import SolutionGenerator

In [4]:
data = ScheduleConfiguration.from_data(config)
generator = SolutionGenerator(data)

profiler = cProfile.Profile()
profiler.enable()
solution = generator.generate_solution()
profiler.disable()

Loading configuration...
Configuration loaded successfully.
Creating Genetic Local Search Algorithm from configuration
Population Size:  25
Max Iteration:  500
Creating FitnessManager with fitness functions:  [Fitness(name=GroupAssignmentConflictFitness), Fitness(name=AssistantDistributionFitness)]
Elitism:  Selection(name=ElitismSelection)
Configured selection functions:  [Selection(name=RouletteWheelSelection), Selection(name=ElitismSelection)]
Configuring crossover operator:  [Crossover(name=SinglePointCrossover)]
Configuring mutation operator:  [Mutation(name=SwapMutation)]
Configuring repair operator:  [Repair(name=RepairTimeSlot)]
Creating FitnessManager with fitness functions:  [Fitness(name=GroupAssignmentConflictFitness), Fitness(name=AssistantDistributionFitness)]
Creating NeighborhoodManager with neighborhood: Neighborhood(name=RandomSwapNeighborhood)
Creating Simulated Annealing
Local Search Algorithm:  Search(name=SimulatedAnnealing)
No available time slot for group id: 19

In [6]:
result = {
    "fitness": solution.best_fitness,
    "time_elapsed": solution.time_elapsed,
}

print(result)

{'fitness': 0, 'time_elapsed': 40.70026516914368}


In [23]:
profiler.print_stats(sort="cumtime")

         47013622 function calls (46952086 primitive calls) in 31.299 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000   31.300   15.650 interactiveshell.py:3472(run_code)
        2    0.000    0.000   31.300   15.650 {built-in method builtins.exec}
        1    0.000    0.000   31.300   31.300 926877854.py:1(<module>)
        1    0.000    0.000   31.300   31.300 solution_generator.py:42(generate_solution)
        1    0.002    0.002   31.204   31.204 genetic_local_search.py:34(run)
        4    0.005    0.001   30.576    7.644 simulated_annealing.py:52(__call__)
        4    0.442    0.111   30.571    7.643 simulated_annealing.py:55(run)
      255    0.057    0.000   22.596    0.089 simulated_annealing.py:122(calculate_fitness)
    25629    0.040    0.000   12.887    0.001 manager.py:20(__call__)
    25629    0.048    0.000   12.836    0.001 manager.py:22(<listcomp>)
    25596    0.027    0.00